In [ ]:
%%capture
!pip install xstac

In [1]:
# List all zarr data in test-data
import pystac
from pystac_client import Client
import s3fs
import sys; sys.path.append('..');
import helpers.eodc_hub_role as eodc_hub_role
import helpers.s3helpers as s3helpers
import helpers.stac as stachelpers

In [39]:
credentials = eodc_hub_role.fetch_and_set_credentials()
bucket = 'nasa-eodc-data-store'
prefix = "test-data"
s3_fs = s3fs.S3FileSystem(
    key=credentials['AccessKeyId'],
    secret=credentials['SecretAccessKey'],
    token=credentials['SessionToken'], 
    anon=False
)

In [52]:
# Create a new STAC Catalog
stac_catalog = pystac.Catalog(
    id='eodc-data-store-for-zarr-viz',
    description='EODC Data Store for Zarr Viz'
)

In [41]:
suffixes = [".zarr", ".json"]
matching_paths = s3helpers.list_s3_paths(bucket, prefix, suffixes)

for path in matching_paths:
    print(path)

nasa-eodc-data-store/test-data/cmip6-kerchunk/combined_CMIP6_daily_GISS-E2-1-G_tas_kerchunk.json
nasa-eodc-data-store/test-data/cmip6-zarr/365_262_262_CMIP6_daily_GISS-E2-1-G_tas.zarr
nasa-eodc-data-store/test-data/cmip6-zarr/600_1440_1_CMIP6_daily_GISS-E2-1-G_tas.zarr
nasa-eodc-data-store/test-data/cmip6-zarr/600_1440_29_CMIP6_daily_GISS-E2-1-G_tas.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat1024_lon2048.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat1448_lon2896.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat2048_lon4096.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat2896_lon5792.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat4096_lon8192.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat512_lon1024.zarr
nasa-eodc-data-store/test-data/fake-data/single_chunk/store_lat724_lon1448.zarr
nasa-eodc-data-store/test-data/fake-data/with_chunks/store_lat1448_lon2896.zar

In [42]:
stac_collection = stachelpers.generate_stac(f's3://{matching_paths[1]}')#, reference=True)
stac_collection

/home/jovyan/tile-benchmarking/02-generate-stac/../titiler_xarray/titiler/xarray/reader.py:45: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  return xarray.open_dataset(src_path, **xr_open_args)


<Collection id=365_262_262_CMIP6_daily_GISS-E2-1-G_tas.zarr>

In [53]:
stac_collection.links = [pystac.Link(target="collection.json", rel="self")]

In [54]:
stac_catalog.add_child(stac_collection)

<Link rel=child target=<Collection id=365_262_262_CMIP6_daily_GISS-E2-1-G_tas.zarr>>

In [55]:
stac_catalog

<Catalog id=eodc-data-store-for-zarr-viz>

In [56]:
stac_catalog.normalize_and_save(root_href="stac")

In [57]:
stac_catalog.save(pystac.CatalogType.SELF_CONTAINED)

In [58]:
STAC_API_URL = "stac/catalog.json"
catalog = Client.open(STAC_API_URL)

In [60]:
collections = list(catalog.get_collections())
for collection in sorted(collections, key=lambda x: x.title):
    print(collection.id)

365_262_262_CMIP6_daily_GISS-E2-1-G_tas.zarr
